In [1]:
import FFTW, Interpolations
import DataFrames, CSV

In [2]:
function feller_cond(kappa, theta, sigma)
    return 2*kappa*theta > sigma^2
end

feller_cond (generic function with 1 method)

In [3]:
theta = 0.132328; kappa = 10.980797; sigma = 1.7; rho = -0.351560; v0 = 0.065690
spot, K = 1.0, 600.0/659.37

alp = 0.75
s = log(spot)
k = log(K)

save_freq = 500

500

In [4]:
@show fw = FFTW.fft([1,2,3,4,5,6])
@show bw = FFTW.ifft(fw)

fw = FFTW.fft([1, 2, 3, 4, 5, 6]) = Complex{Float64}[21.0 + 0.0im, -3.0 + 5.196152422706632im, -3.0 + 1.7320508075688772im, -3.0 + 0.0im, -3.0 - 1.7320508075688772im, -3.0 - 5.196152422706632im]
bw = FFTW.ifft(fw) = Complex{Float64}[1.0 + 0.0im, 2.0 + 0.0im, 3.0 + 0.0im, 4.0 + 0.0im, 5.0 + 0.0im, 6.0 + 0.0im]


6-element Array{Complex{Float64},1}:
 1.0 + 0.0im
 2.0 + 0.0im
 3.0 + 0.0im
 4.0 + 0.0im
 5.0 + 0.0im
 6.0 + 0.0im

In [5]:
function get_Heston_chf(x, u, v0, r, rho, ka, th, sig, T)
    gam = sqrt(sig^2*u*(u+im) + (ka-im*rho*sig*u)^2)
    half_gam_T = gam*T/2.0
    coth = 1.0/tanh(half_gam_T)
    log_numerator = ka*th*T*(ka-im*rho*sig*u)/sig^2 + im*u*(T*r+x) - (u*(u+im)*v0)/(gam*coth + ka - im*rho*sig*u)
    numerator = exp(log_numerator)
    denominator = (cosh(half_gam_T) + (ka-im*rho*sig*u)/gam*sinh(half_gam_T))^(2.0*ka*th/sig^2)
    return numerator/denominator
    
end

function get_damped_Heston_chf(x, v, v0, r, rho, ka, th, sig, T, alp=0.75)
    chf_inp = v - (alp+1.0)*im
    numerator = exp(-r*T) * get_Heston_chf(x, chf_inp, v0, r, rho, ka, th, sig, T)
    denominator = alp^2 + alp - v^2 +im*(2.0*alp +1.0)*v
    return numerator / denominator    
end

function get_Heston_fft_call(k, x, v0, r, rho, kappa, theta, sigma, T, alp, N=2^12, dk=0.01)# 2048*5
    u_list = collect(0:N-1) 
    dv = 2.0*pi/(N*dk)
    k_list = [x + dk*u_elem - N*dk/2.0 for u_elem in u_list]
    v_list = collect(0:N-1)*dv
    x_list = zeros(Complex{Float64},length(v_list))

    for (j,vj) in enumerate(v_list) # j=0 to N-1
        jj = j-1 # 0-based indexing
        if jj==0
            kroneker_delta = 1.0
        else
            kroneker_delta = 0.0        
        end
        simpson_coeff = dv/3.0*(3.0 + (-1)^jj - kroneker_delta)
        x_list[j] = exp(im*(N/2.0*dk-x)*vj)*simpson_coeff*get_damped_Heston_chf(x, vj, v0, r, rho, kappa, theta, sigma, T, alp)
    end
    fft_prices = exp(-alp*k)/pi*real(FFTW.fft(x_list))

    # interpolate results
#    println(k_list)
#    println(fft_prices[end-5:end])
    return Interpolations.LinearInterpolation(exp.(k_list),fft_prices)(exp(k)) # fft_price

end


get_Heston_fft_call (generic function with 3 methods)

In [6]:
r = 0.0
T = 1.0
@show get_Heston_chf(s, 1.0, v0, r, rho, kappa, theta, sigma, T) # analytic chf  (0.9345074136306541-0.055431779426290094j)
@show get_damped_Heston_chf(s, 1.0, v0, r, rho, kappa, theta, sigma, T)
@show get_Heston_fft_call(k, s, v0, r, rho, kappa, theta, sigma, T, alp,2^15,1e-2)

get_Heston_chf(s, 1.0, v0, r, rho, kappa, theta, sigma, T) = 0.9345074136306533 - 0.055431779426290094im
get_damped_Heston_chf(s, 1.0, v0, r, rho, kappa, theta, sigma, T) = 0.10800278041809996 - 0.3916096746416378im
get_Heston_fft_call(k, s, v0, r, rho, kappa, theta, sigma, T, alp, 2 ^ 15, 0.01) = 0.18431595807352924


0.18431595807352924

# Generate Dataset

In [10]:
#num_grid_1d = 3
#num_grid_1d = 5
num_grid_1d = 10
alp = 0.75
C_price_bounds = [1e-3, 0.5]
k_list = range(-0.2,0.2,length=num_grid_1d)
logS0 = 0.0 # log-spot. Assume S0: 1
dlogS0 = log(1.00001)
dS0 = exp(dlogS0) - exp(logS0)
T_list = [0.25,0.5,0.75,1.0,2.0]
v0_list = range(1e-2,sqrt(0.2),length=num_grid_1d) 
r = 0.0
rho_list = range(-0.8,0.8,length=num_grid_1d) 
kappa_list = range(1e-3,20.0,length=num_grid_1d) 
theta_list = range(1e-3,10.0,length=num_grid_1d) 
sigma_max = sqrt(2.0*maximum(kappa_list)*maximum(theta_list)) 
sigma_list = range(1e-3,0.8,length=num_grid_1d) 

pd_dict = Dict()
pd_dict["k"] = []; pd_dict["T"] = []; pd_dict["v0"] = []; pd_dict["rho"] = []
pd_dict["kappa"] = []; pd_dict["theta"] = []; pd_dict["sigma"] = []; pd_dict["C_price"] = []
pd_dict["delta"] = []
violation_count = 0
count = 0

0

In [11]:
for kappa in kappa_list
    for theta in theta_list
        for sigma in sigma_list
            f_cond = feller_cond(kappa, theta, sigma)
            if f_cond == true
                for k in k_list
                    for T in T_list
                        for v0 in v0_list
                            for rho in rho_list
#                                println((k, logS0, v0, r, rho, kappa, theta, sigma, T, alp, 2^15, 1e-2))
                                C_price = get_Heston_fft_call(k, logS0, v0, r, rho, kappa, theta, sigma, T, alp, 2^15, 1e-2)
#                                println(C_price, " ", isnan(C_price))
                                if (C_price <= C_price_bounds[1]) | (C_price >= C_price_bounds[2]) | isnan(C_price)
                                    continue
                                end
                                count += 1
                                C_price_p = get_Heston_fft_call(k,logS0+dlogS0, v0, r, rho, kappa, theta, sigma,T,alp,2^15,1e-2)                                
#                                println(C_price_p, " ", isnan(C_price_p))
                                append!(pd_dict["delta"],(C_price_p-C_price)/dS0) # delta of call 
                                append!(pd_dict["k"],k)
                                append!(pd_dict["T"],T)
                                append!(pd_dict["v0"],v0)
                                append!(pd_dict["rho"],rho)
                                append!(pd_dict["kappa"],kappa)
                                append!(pd_dict["theta"],theta)
                                append!(pd_dict["sigma"],sigma)
                                append!(pd_dict["C_price"],C_price)
                                if count % save_freq == 0
                                   println("Intermediate save ", count)
                                df = DataFrames.DataFrame(pd_dict)
                                CSV.write("Heston_Julia.csv",df)
                                end                                
                            end
                        end
                    end
                end
            else
                violation_count += 1
            end
        end
    end
end

df = DataFrames.DataFrame(pd_dict)
CSV.write("Heston_Julia.csv",df)


Intermediate save 500
Intermediate save 1000
Intermediate save 1500
Intermediate save 2000
Intermediate save 2500
Intermediate save 3000
Intermediate save 3500
Intermediate save 4000
Intermediate save 4500
Intermediate save 5000
Intermediate save 5500
Intermediate save 6000
Intermediate save 6500
Intermediate save 7000
Intermediate save 7500
Intermediate save 8000
Intermediate save 8500
Intermediate save 9000
Intermediate save 9500
Intermediate save 10000
Intermediate save 10500
Intermediate save 11000
Intermediate save 11500
Intermediate save 12000
Intermediate save 12500
Intermediate save 13000
Intermediate save 13500
Intermediate save 14000
Intermediate save 14500
Intermediate save 15000
Intermediate save 15500
Intermediate save 16000
Intermediate save 16500
Intermediate save 17000
Intermediate save 17500
Intermediate save 18000
Intermediate save 18500
Intermediate save 19000
Intermediate save 19500
Intermediate save 20000
Intermediate save 20500
Intermediate save 21000
Intermediate

"Heston_Julia.csv"

In [9]:
120%3

0